In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb

from scipy.optimize import minimize

In [ ]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 데이터 로드

In [ ]:
# 로컬 버전

# data_dir = Path('../input/dankook')
# feature_dir = Path('../output/feature')
# val_dir = Path('../output/oof_pred')
# test_dir = Path('../output/test_pred')
# sub_dir = Path('../output/sub')


# train_file = data_dir / 'train.csv'
# test_file = data_dir / 'test.csv'
# sample_file = data_dir / 'sample_submission.csv'

In [ ]:
# 코렙 

from google.colab import drive
drive.mount('/content/drive')

data_dir = Path('/content/drive/My Drive/Colab Notebooks/input/dankook')
feature_dir = Path('/content/drive/My Drive/Colab Notebooks/output/feature')
val_dir = Path('/content/drive/My Drive/Colab Notebooks/output/oof_pred')
test_dir = Path('/content/drive/My Drive/Colab Notebooks/output/test_pred')
sub_dir = Path('/content/drive/My Drive/Colab Notebooks/output/sub')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
algorithm_name = 'xgbcv'
feature_name = 'stacking2'
model_name = f'{algorithm_name}_{feature_name}_4'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_target_file = feature_dir / f'feature_target.csv'

stacking1_oof_pred_file = val_dir / f'{model_name}_oof_pred.csv'
stacking1_test_pred_file = test_dir / f'{model_name}_test_pred.csv'
stacking1_submission_file = sub_dir / f'{model_name}_submission.csv'

In [ ]:
SEED = 2020
num_class = 3
n_splits = 5
target_column = 'class'

# Stacking Feature 생성

In [ ]:
def load_data(model_names, oof_list, test_list, feature_names=None):
    for model in model_names:
        oof_list.append(np.loadtxt(val_dir / f'{model}_oof_pred.csv',delimiter=','))
        test_list.append(np.loadtxt(test_dir / f'{model}_test_pred.csv',delimiter=','))
        if feature_names != None:
            feature_names += {f'{model}_class0',f'{model}_class1',f'{model}_class2'}

In [ ]:
all_oof = []
all_test = []
feature_names = []

model_names = ['xgbcv_stacking1_8',
               'lgbmcv_stacking1_5',
               'rfcv_stacking1_1','lrcv_stacking1_1','gbcv_stacking1_1','etscv_stacking1_1','adacv_stacking1_2']
load_data(model_names,all_oof, all_test, feature_names)

all_oof = np.column_stack(all_oof)
all_test = np.column_stack(all_test)
all_oof.shape, all_test.shape, feature_names

((319923, 21),
 (80000, 21),
 ['xgbcv_stacking1_8_class1',
  'xgbcv_stacking1_8_class2',
  'xgbcv_stacking1_8_class0',
  'lgbmcv_stacking1_5_class1',
  'lgbmcv_stacking1_5_class2',
  'lgbmcv_stacking1_5_class0',
  'rfcv_stacking1_1_class1',
  'rfcv_stacking1_1_class0',
  'rfcv_stacking1_1_class2',
  'lrcv_stacking1_1_class2',
  'lrcv_stacking1_1_class0',
  'lrcv_stacking1_1_class1',
  'gbcv_stacking1_1_class1',
  'gbcv_stacking1_1_class2',
  'gbcv_stacking1_1_class0',
  'etscv_stacking1_1_class0',
  'etscv_stacking1_1_class1',
  'etscv_stacking1_1_class2',
  'adacv_stacking1_2_class0',
  'adacv_stacking1_2_class1',
  'adacv_stacking1_2_class2'])

In [ ]:
y = pd.read_csv(feature_target_file, index_col=0, usecols=['id',target_column]).values.flatten()
y.shape

(319923,)

# Stacking

- 각 oof마다 fold별로 logloos의 변동이 있으므로 최대한 정보를 뽑아내고자 스태킹을 함.

In [ ]:
# Xgboost
stacking_params = {
    'n_jobs' : -1, 
    'n_estimators': 100,
    'eval_metric': 'mlogloss',
    'eta': 0.3, # learning_rate
    'booster': 'gbtree',
    'objective': 'multi:softprob',
    'num_class': 3,
    'random_state': 2020,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'subsample': 1.0,
    'min_child_weight': 1,
    'max_depth': 3,
    'gamma': 0.5,
    'colsample_bytee': 0.6
}

In [ ]:
rmlogloss = []

stk_oof_pred = np.zeros((all_oof.shape[0],num_class))
stk_test_pred = np.zeros((all_test.shape[0],num_class))

kFold = StratifiedKFold(n_splits=n_splits, random_state=2020, shuffle=True)
for fold, (trn_idx, val_idx) in enumerate(kFold.split(all_oof,y)):
    X_train, X_val = all_oof[trn_idx], all_oof[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dval = xgb.DMatrix(X_val, label=y_val)
    watchlist = [(dtrain,'train'), (dval, 'val')]

    xgb_clf = xgb.train(stacking_params, dtrain, 5000, evals=watchlist, early_stopping_rounds=50, verbose_eval=5000)

    dtest = xgb.DMatrix(all_test)
    stk_test_pred += xgb_clf.predict(dtest) / n_splits
    stk_oof_pred[val_idx] = xgb_clf.predict(dval)
    mlogloss.append(xgb_clf.best_score)

print('mean logloss= ', np.mean(mlogloss))

[0]	train-mlogloss:0.756633	val-mlogloss:0.757894
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[53]	train-mlogloss:0.147645	val-mlogloss:0.15129

[0]	train-mlogloss:0.756862	val-mlogloss:0.756673
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[52]	train-mlogloss:0.14811	val-mlogloss:0.149352

[0]	train-mlogloss:0.75683	val-mlogloss:0.756955
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[49]	train-mlogloss:0.148243	val-mlogloss:0.149787

[0]	train-mlogloss:0.756725	val-mlogloss:0.756164
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss has

# Ensemble

- cv score가 안정적이여서 Stacking 예측값와 함께 가중치 최적화함
- 가중치 최적화에는 scipy의 minimize 사용

In [ ]:
def log_loss_func(weights):
    final_prediction = 0
    for weight, prediction in zip(weights, oof_predictions):
        final_prediction += weight*prediction

    return log_loss(y, final_prediction)

In [ ]:
oof_predictions = []
test_predictions = []

model_names = ['xgbcv_stacking1_8',
               'lgbmcv_stacking1_5',
               'rfcv_stacking1_1','lrcv_stacking1_1','gbcv_stacking1_1','etscv_stacking1_1','adacv_stacking1_2']
load_data(model_names,oof_predictions, test_predictions)

oof_predictions.append(stk_oof_pred)
test_predictions.append(stk_test_pred)
len(oof_predictions), oof_predictions, len(test_predictions), test_predictions

(8, [array([[9.99997258e-01, 1.60694617e-06, 1.18237051e-06],
         [2.09188089e-04, 4.82893109e-01, 5.16897738e-01],
         [9.99983549e-01, 9.95367463e-06, 6.46004582e-06],
         ...,
         [9.99998212e-01, 1.24602468e-06, 5.75672857e-07],
         [9.99998093e-01, 1.25574206e-06, 6.01579245e-07],
         [9.99991179e-01, 4.59897137e-06, 4.27725490e-06]]),
  array([[9.99086382e-01, 2.42415838e-04, 6.71201862e-04],
         [1.09544468e-02, 4.73873586e-01, 5.15171968e-01],
         [9.96615337e-01, 6.53947760e-04, 2.73071491e-03],
         ...,
         [9.99093832e-01, 2.42417476e-04, 6.63750253e-04],
         [9.99072930e-01, 2.46704197e-04, 6.80366187e-04],
         [9.99064370e-01, 2.47872834e-04, 6.87757070e-04]]),
  array([[1.  , 0.  , 0.  ],
         [0.01, 0.53, 0.46],
         [1.  , 0.  , 0.  ],
         ...,
         [1.  , 0.  , 0.  ],
         [1.  , 0.  , 0.  ],
         [1.  , 0.  , 0.  ]]),
  array([[9.99709807e-01, 1.52406280e-04, 1.37786811e-04],
        

In [ ]:
best_scores = []
weights = []

iteration = 100 # 여러 번 구한 가중치 값 중 Best score를 선택
for i in tqdm(range(iteration)):
    starting_values = np.random.uniform(size=len(oof_predictions))
    bounds = [(0,1)]*len(oof_predictions)
    cons = ({'type':'eq','fun':lambda w: 1-sum(w)})

    res = minimize(log_loss_func,
                   starting_values,
                   method = 'SLSQP',
                   bounds = bounds,
                   constraints = cons,
                   options={'maxiter': 500})
    best_scores.append(res['fun'])
    weights.append(res['x'])

weights = weights[np.argmin(best_scores)]
print('\n')
print('Ensemble score: {}'.format(np.min(best_scores)))
print('Best Weights: {}'.format(weights))

100%|██████████| 100/100 [29:10<00:00, 17.51s/it]



Ensemble score: 0.1501608400311149
Best Weights: [1.78565011e-01 5.38035328e-18 3.64409811e-02 0.00000000e+00
 2.46378597e-01 7.19888509e-02 2.64791644e-02 4.40147396e-01]


In [ ]:
final_prediction = 0

# 가중치 적용
for weight, p in zip(weights, test_predictions):
    final_prediction += weight*p

print(final_prediction)

[[1.30311509e-04 1.58332379e-01 8.41537308e-01]
 [9.99830760e-01 7.11806572e-05 9.80464031e-05]
 [1.17949615e-04 8.68618712e-02 9.13020184e-01]
 ...
 [9.99912543e-01 2.49936904e-05 6.24657838e-05]
 [1.05287565e-04 5.09864482e-02 9.48908268e-01]
 [9.99914642e-01 2.50432935e-05 6.03127879e-05]]


# 제출 파일 및 기타 파일 생성

In [ ]:
# # submission 파일 생성

# sub = pd.read_csv(sample_file)
# sub[target_column] = np.argmax(stk_test_pred, axis=1)
# sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

# submission 파일 생성

sub = pd.read_csv(sample_file)
sub[target_column] = np.argmax(final_prediction, axis=1)
sub.to_csv(stacking1_submission_file, index=False, encoding='utf-8-sig')

In [ ]:
# stcking1_oof_pred 파일 생성

np.savetxt(stacking1_oof_pred_file, stk_oof_pred, fmt='%.18f',delimiter=',')

In [ ]:
# stacking1_test_pred 파일 생성

np.savetxt(stacking1_test_pred_file, stk_test_pred, fmt='%.18f', delimiter=',')

In [ ]:
0.938